# Dish Price Calculator
calculate the price of dish based on live price of the incredients 

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import google.generativeai as genai
from typing import List, Dict
from vertexai import init
import json
import gradio as gr
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)


In [2]:
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Volumes/SD-Card/LLM/LLM-Projects/service-account-key.json"


# connect to gemini
genai.configure(api_key = api_key)

In [3]:
response = ""

In [4]:
# use beautiful soup for web-scraping price from the website

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Pricing:
    item_prices : Dict[str, float]
    url= "https://fcainfoweb.nic.in/Reports/DB/Dailyprices.aspx"

    def __init__(self):
        self.item_prices = {}
        self.scrape_data()


    def scrape_data(self):
        try:
            global response
            if response == "":
                response = requests.get(self.url, headers=headers)
                print(response)
                if response.status_code != 200:
                    print(f"Failed to retrieve the URL. Status code: {response.status_code}")
                    return
            print("request-----")
            soup = BeautifulSoup(response.content, 'html.parser')
            body = response.content
            print(body)
            title = soup.title.string if soup.title else "No title found!"
            print(title)
            allTrs = soup.find_all('tr')
            for i in range(1, len(allTrs)):
                if(i <4):
                    continue
                # Extract descendants and convert to text
                descendants = list(allTrs[i].descendants)
                if len(descendants) >= 2:
                    text1 = descendants[0].text if hasattr(descendants[0], 'text') else str(descendants[0])
                    text2 = descendants[2].text if hasattr(descendants[2], 'text') else str(descendants[1])
                    self.item_prices[text1] = float(text2)
                    if text1 == "Tomato":
                        break
                else:
                    print("Not enough descendants in this row")
        except Exception as e:
            print(f"An error occurred while scraping: {e}")
        

        
pricing = Pricing()



<Response [200]>
request-----
b'\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" >\r\n<head id="Head1"><title>\r\n\tWeekly Avg. Report Data Commoditywise\r\n</title><meta http-equiv="Pragma" content="no-cache" /><meta http-equiv="Expires" content="-1" /><meta http-equiv="CACHE-CONTROL" content="NO-CACHE" />\r\n\r\n<script type = "text/javascript">\r\n\r\n    function PrintPanel() {\r\n\r\n        var panel = document.getElementById("Panel1");\r\n        var printWindow = window.open(\'\', \'\', \'\');\r\n\r\n        printWindow.document.write(panel.innerHTML);\r\n        printWindow.document.close();\r\n        setTimeout(function () {\r\n            //printWindow.print();\r\n        }, 500);\r\n        return false;\r\n    }\r\n    </script>\r\n</head>\r\n<body>\r\n    <form name="form1" method="post" action="./Dailyprices.aspx" id="form1">\r\n<div

In [20]:
pricing.item_prices

{'Rice': 43.0,
 'Wheat': 32.82,
 'Atta (Wheat)': 37.71,
 ' Gram Dal(Bharat Dal Adjusted)': 87.34,
 'Gram Dal': 92.17,
 'Tur/Arhar Dal': 149.2,
 'Urad Dal': 122.69,
 'Moong Dal': 113.39,
 'Masoor Dal': 89.07,
 'Sugar': 44.79,
 'Milk @': 58.58,
 'Groundnut Oil (Packed)': 193.43,
 'Mustard Oil (Packed)': 168.38,
 'Vanaspati (Packed)': 147.9,
 'Soya Oil (Packed)': 143.65,
 'Sunflower Oil (Packed)': 154.31,
 'Palm Oil (Packed)': 135.15,
 'Gur': 54.61,
 'Tea Loose': 270.82,
 'Salt Pack (Iodised)': 21.58,
 'Potato': 28.84,
 'Onion': 37.63,
 'Tomato': 27.2}

candidates {
  content {
    role: "model"
    parts {
      text: "I\'m sorry, I can\'t give you the price of dal chawal. I can only access information on individual ingredients. Is there anything else I can help you with? \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.055908203125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.052734375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0927734375
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.036865234375
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.054931640625
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.029296875
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.1201171875
    s

In [5]:
system_message = "You are a helpful caterer assistant for a company called ziti."
system_message += "Give short, courteous answers"
system_message += "Always be accurate. If you don't know the answer, say so."

In [6]:
init(project="flutter-shop-f2274", location="asia-south1")
model = GenerativeModel("gemini-1.5-pro-001", system_instruction=system_message)

In [25]:

price_calc_func = FunctionDeclaration(
    name="get_price_data",
    description="Call this whenever you need to know the price of a dish like if the user asks: 'what is the price of chole bhature?'",
    parameters={
        "type": "object",
        "properties": {
            "dish_name": {
                "type": "string",
                "description": "name of the dish requested by user",
            }
        },
        "required": [
            "dish_name"
        ]
    },
)

In [26]:
price_calculator_tool = Tool(
    function_declarations=[price_calc_func]
)

In [27]:
msg_history = []

def get_price_data(dish_name, prompt):
    print("Fetching dish data---")
    recipe = get_recipe(dish_name)
    total_price = 0
    ingredient_prices = {}
    print("Fetching price data---")
    for item in recipe:
        if (item['ingredient_name'] not in pricing.item_prices.keys()):
            print(f"No price data available for {item['ingredient_name']}")
            continue
        price = pricing.item_prices[item['ingredient_name']]
        amount_in_grams = float(item['ingredient_quantity'])
        net_price = price*0.001*amount_in_grams
        ingredient_prices[item['ingredient_name']] = net_price
        total_price += net_price
    final_price = {'total_price': round(total_price, 2), 'ingredient_prices': ingredient_prices}

    #pass back the response
    respond_back = model.generate_content(
    [
        Content(role="user", parts=[
            Part.from_text(prompt + """Give your answer in steps with lots of detail
                and context, including total price of dish and breakdown of price of each ingredient. Price is in indian rupees. 
                You are providing cost price detail of dish for one person to caterer so that he can price it accordingly to customers."""),
        ]),
        Content(role="function", parts=[
            Part.from_dict({
                "function_call": {
                    "name": "get_price_data",
                }
            })
        ]),
        Content(role="function", parts=[
            Part.from_function_response(
                name="get_price_data",
                response={
                    "content": final_price,
                }
            )
        ]),
        ],
        tools=[price_calculator_tool],
    )
    return respond_back.candidates[0].content.parts[0].text



Calculate the ingredients and their quantities for one person

In [36]:
system_message2 = f"You are provided a Dish name.\
You are able to find out the ingredients and quantity of each ingredient (in grams) to prepare that dish.\
Only include ingredients from this list: {list(pricing.item_prices.keys())}"
system_message2 += "You should respond in JSON as in this example but Don't include json term in output:"
system_message2 += """
{
    "ingredients": [
        {"ingredient_name": "Tomato", "ingredient_quantity": "100"},
    ]
}
"""
def get_recipe(dish_name):
    print(f"Received Dish name : {dish_name}")
    model2 = GenerativeModel("gemini-1.5-pro-001", system_instruction=system_message2)
    output = model2.generate_content(dish_name)
    print(output)
    recipe  = json.loads(output.candidates[0].content.parts[0].text)
    print(recipe['ingredients'])
    return recipe['ingredients']
    

In [37]:

def chat(message, history):
    messages = history + [{"role": "user", "content": message}]
    response1 = model.generate_content(
        str(messages),
        tools=[price_calculator_tool],
    )
    if("function_call" in str(response1)):
        response1.candidates[0].content.parts[0].function_call.name == "get_price_data"
        dish_name = response1.candidates[0].content.parts[0].function_call.args['dish_name']
        return get_price_data(dish_name, message)
    return response1.candidates[0].content.parts[0].text

In [39]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


Fetching dish data---
Received Dish name : dal chawal
candidates {
  content {
    role: "model"
    parts {
      text: "{\n    \"ingredients\": [\n        {\"ingredient_name\": \"Rice\", \"ingredient_quantity\": \"100\"},\n        {\"ingredient_name\": \"Tur/Arhar Dal\", \"ingredient_quantity\": \"50\"},\n        {\"ingredient_name\": \"Onion\", \"ingredient_quantity\": \"30\"},\n        {\"ingredient_name\": \"Tomato\", \"ingredient_quantity\": \"20\"},\n        {\"ingredient_name\": \"Groundnut Oil (Packed)\", \"ingredient_quantity\": \"25\"},\n        {\"ingredient_name\": \"Salt Pack (Iodised)\", \"ingredient_quantity\": \"5\"}\n    ]\n}\n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.2578125
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.1708984375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0